## EE 461P: Data Science Principles  
### Term Project
### Blog Due: May 13, 2021 
### Presentation May 4, 2021





In [41]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
import os, sys, re
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import plot_roc_curve 
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.naive_bayes import BernoulliNB

#Load dataset from csv file
df = pd.read_csv("PS_20174392719_1491204439457_log.csv")
df.drop(columns=['isFlaggedFraud'])
for col in ["type","nameOrig","nameDest"]:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.codes
print(df.head())
fraud_rows = df[df['isFraud']==1].copy()
not_fraud_rows = df[df['isFraud']==0].copy()
X = df
y = df.pop("isFraud")
X_tr, X_te, y_tr, y_te = train_test_split(X,y, test_size=0.25, random_state = 42)
bnb = BernoulliNB(fit_prior=True)

f_y = fraud_rows.pop("isFraud")
f_x = fraud_rows
nf_y = not_fraud_rows.pop("isFraud")
nf_x = not_fraud_rows
print("data loaded")

   step  type    amount  nameOrig  oldbalanceOrg  newbalanceOrig  nameDest  \
0     1     3   9839.64    757869       170136.0       160296.36   1662094   
1     1     3   1864.28   2188998        21249.0        19384.72   1733924   
2     1     4    181.00   1002156          181.0            0.00    439685   
3     1     1    181.00   5828262          181.0            0.00    391696   
4     1     3  11668.14   3445981        41554.0        29885.86    828919   

   oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0             0.0             0.0        0               0  
1             0.0             0.0        0               0  
2             0.0             0.0        1               0  
3         21182.0             0.0        1               0  
4             0.0             0.0        0               0  
data loaded


In [42]:
bnb.fit(X_tr,y_tr)
print("Model Accuracy on All Test Data: {}".format(bnb.score(X_te,y_te)))
print("Model Accuracy for predicting true fraud: {}".format(bnb.score(f_x,f_y)))
print("Model Accuracy for predicting true not fraud: {}".format(bnb.score(nf_x,nf_y)))
print(sum(bnb.predict(X_te)))
print(sum(y_te))

Model Accuracy on All Test Data: 0.9987181381254892
Model Accuracy for predicting true fraud: 0.0019481310118105442
Model Accuracy for predicting true not fraud: 1.0
6
2045
